# 👩‍🚀**<span style="color:#4527A0;"> Spaceship Titanic with</span> <span style="color:#F44336;"> LGBMClassifier </span>🌠**

# <span style="color:#4527A0;"> Project </span> 🔥
- 강의명 : 2022년 K-디지털 직업훈련(Training) 사업 - AI데이터플랫폼을 활용한 빅데이터 분석전문가 과정
- 교과목명 : 빅데이터 분석 및 시각화, AI개발 기초, 인공지능 프로그래밍
- 프로젝트 주제 : Spaceship Titanic 데이터를 활용한 탑승유무 분류모형 개발
- 프로젝트 마감일 : 2022년 4월 12일 화요일
- 강사명 : 정지훈 강사
- 수강생명 : 이가영

# <span style="color:#4527A0;"> Table of content </span> 📋

- [Load data](#Load-Data)
- [EDA](#EDA)
- [Preprocessing](#Preprocessing)
- [Pipeline](#Pipeline)
- [Model](#Model)

# Load Data
- 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

- 데이터 불러오기

In [2]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

In [3]:
train.shape, test.shape, submission.shape

((8693, 14), (4277, 13), (4277, 2))

## train Data

In [4]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


## Test Data

In [8]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [10]:
test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [11]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


## EDA

- 행성 출발지(**HomePlanet**)와 행성 도착치(**Destination**)의 비율을 알아본다.

In [12]:
# fig = go.Figure()
from plotly.subplots import make_subplots

c = ['HomePlanet','Destination']
fig = make_subplots(rows=1, cols=2,shared_yaxes=True)
    
fig.add_trace(go.Scatter( x=train['HomePlanet'].value_counts().index, y=train['HomePlanet'].value_counts().values,
                         mode='markers',name = 'HomePlanet',
        marker=dict(
            sizemode = 'diameter',
            sizeref = 25,
            size = train['HomePlanet'].value_counts().values)),1,1)
    
fig.add_trace(go.Scatter( x=train['Destination'].value_counts().index, y=train['Destination'].value_counts().values,
                         mode='markers', name = 'Destination',
        marker=dict(
            sizemode = 'diameter',
            sizeref = 25,
            size = train['Destination'].value_counts().values)),1,2)


fig.update_layout(    title='<b>Planets</b>', 
        title_x=0.5,
        titlefont=dict(size =20, color='black', family='Space Mono'),
        plot_bgcolor='rgba(0,0,0,0)' )
fig.show()

---
시각화 추가

---

# Preprocessing

### PassengerId

- PassengerId의 앞자리 4개를 이용해 가족 그룹 Id 생성 -> 컬럼 Group으로 데이터 값 채우기

In [13]:
train['Group'] = train['PassengerId'].apply(lambda x : str(x[:4]))
test['Group'] = test['PassengerId'].apply(lambda x : str(x[:4]))
train[['PassengerId','Group']].head()

,PassengerId,Group
0,0001_01,0001
1,0002_01,0002
2,0003_01,0003
3,0003_02,0003
4,0004_01,0004


In [14]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Group            object
dtype: object

PassengerId의 앞의 4자리가 잘 분리 되었고 Object 형태로 저장이 되었다.

- Group을 이용해 가족 구성원 수를 구한다.

In [15]:
train['Family'] = train.groupby('Group')['Group'].transform('count')
test['Family'] = test.groupby('Group')['Group'].transform('count')
train[['Group','Family']].sort_values(['Group']).head()

,Group,Family
0,0001,1
1,0002,1
2,0003,2
3,0003,2
4,0004,1


In [16]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Group            object
Family            int64
dtype: object

In [17]:
train['Family'].describe()

count    8693.000000
mean        2.035546
std         1.596347
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max         8.000000
Name: Family, dtype: float64

가족 구성원의 수는 1인 가족 부터 최대 8인 가족까지 존재한다.

### Cabin 나누기

- cd : Cabin Deck
- cn : Cabin Number
- cs : Cabin Side

In [18]:
train['cd'] = train['Cabin'].str.split('/').str[0]
train['cn'] = train['Cabin'].str.split('/').str[1]
train['cs'] = train['Cabin'].str.split('/').str[2]

test['cd'] = test['Cabin'].str.split('/').str[0]
test['cn'] = test['Cabin'].str.split('/').str[1]
test['cs'] = test['Cabin'].str.split('/').str[2]

# Pipeline

- Pipeline 구축하기 전에 훈련 데이터 세트**와 테스트 데이터 세트로 분리

---

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
label_cols= ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP","cd","cs"]

In [21]:
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] = LabelEncoder().fit_transform(test[col])
    return train, test

In [22]:
train, test = label_encoder(train,test,label_cols)

In [23]:
from sklearn.impute import SimpleImputer

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService","Family","cn"]
STRATEGY = 'median'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print("train_data:\n", train.isnull().sum())
print("---")
print("test_data:\n", test.isnull().sum())

train_data:
 PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Group             0
Family            0
cd                0
cn                0
cs                0
dtype: int64
---
test_data:
 PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin            0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
Group            0
Family           0
cd               0
cn               0
cs               0
dtype: int64


In [24]:
imputer_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP","cd","cs"]
STRATEGY = 'most_frequent'

imputer = SimpleImputer(strategy=STRATEGY)
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

print("train_data:\n", train.isnull().sum())
print("---")
print("test:\n", test.isnull().sum())

train_data:
 PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
Group             0
Family            0
cd                0
cn                0
cs                0
dtype: int64
---
test:
 PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin            0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
Group            0
Family           0
cd               0
cn               0
cs               0
dtype: int64


In [25]:
train = train.drop(['Name','Cabin'],axis = 1)
test = test.drop(['Name','Cabin'],axis = 1)

---

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   int64  
 2   CryoSleep     8693 non-null   int64  
 3   Destination   8693 non-null   int64  
 4   Age           8693 non-null   float64
 5   VIP           8693 non-null   int64  
 6   RoomService   8693 non-null   float64
 7   FoodCourt     8693 non-null   float64
 8   ShoppingMall  8693 non-null   float64
 9   Spa           8693 non-null   float64
 10  VRDeck        8693 non-null   float64
 11  Transported   8693 non-null   bool   
 12  Group         8693 non-null   object 
 13  Family        8693 non-null   float64
 14  cd            8693 non-null   int64  
 15  cn            8693 non-null   float64
 16  cs            8693 non-null   int64  
dtypes: bool(1), float64(8), int64(6), object(2)
memory usage: 1.1+ MB


In [27]:
col = train.drop(['PassengerId','Transported','Group'], axis = 1).columns.tolist()
TARGET = 'Transported'

In [28]:
X = train[col]
Y = train[TARGET]
X_train, y_train, X_val, y_val = train_test_split(X,Y,random_state = 12,test_size = 0.33)

- 모델 적용

---

# 모델

In [29]:
print(train.columns)
print(test.columns)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'Group', 'Family', 'cd', 'cn', 'cs'],
      dtype='object')
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Group',
       'Family', 'cd', 'cn', 'cs'],
      dtype='object')


In [30]:
col = train.drop(['PassengerId','Transported','Group'], axis = 1).columns.tolist()
TARGET = 'Transported'

In [31]:
col

['HomePlanet',
 'CryoSleep',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Family',
 'cd',
 'cn',
 'cs']

In [32]:
X = train[col]
Y = train[TARGET]
X_train, Y_train, X_target, Y_target = train_test_split(X,Y,random_state = 12,test_size = 0.33)

In [33]:
# type_fold_indice = List[Tuple[np.ndarray, np.ndarray]]

idx_fold = []

skf = StratifiedKFold(5, random_state=12, shuffle=True)
for idx_train, idx_valid in skf.split(X = X, y = Y):
    idx_fold.append((idx_train, idx_valid))

In [34]:
'''
lgb_params = {
    'objective' : 'binary',
    'n_estimators' : 100,
    'learning_rate' : 0.05,
    'random_state' : 12,
    'n_jobs' : -1,
    'importance_type' : 'gain',
    'subsample' : 0.9,
    'subsample_freq' : 3,
    'colsample_bytree' : 0.6
}
'''

"\nlgb_params = {\n    'objective' : 'binary',\n    'n_estimators' : 100,\n    'learning_rate' : 0.05,\n    'random_state' : 12,\n    'n_jobs' : -1,\n    'importance_type' : 'gain',\n    'subsample' : 0.9,\n    'subsample_freq' : 3,\n    'colsample_bytree' : 0.6\n}\n"

In [35]:
'''
lgb_params = {
    'objective' : 'binary',
    'n_estimators' : 975,
    'min_child_samples' : 5,
    'max_depth' : 8,
    'learning_rate' : 0.01,
    'random_state' : 12,
    'n_jobs' : -1,
    'importance_type' : 'gain',
    'subsample' : 0.9,
    'subsample_freq' : 3,
    'colsample_bytree' : 0.6,
    'feature_fraction' : 0.6,
    'bagging_freq' : 5, 
    'bagging_fraction' : 0.9
}
'''

"\nlgb_params = {\n    'objective' : 'binary',\n    'n_estimators' : 975,\n    'min_child_samples' : 5,\n    'max_depth' : 8,\n    'learning_rate' : 0.01,\n    'random_state' : 12,\n    'n_jobs' : -1,\n    'importance_type' : 'gain',\n    'subsample' : 0.9,\n    'subsample_freq' : 3,\n    'colsample_bytree' : 0.6,\n    'feature_fraction' : 0.6,\n    'bagging_freq' : 5, \n    'bagging_fraction' : 0.9\n}\n"

In [36]:
lgb_params = {
    'objective' : 'binary',
    'n_estimators' : 975,
    'min_child_samples' : 5,
    'max_depth' : 8,
    'learning_rate' : 0.01,
    'random_state' : 12,
    'n_jobs' : -1,
    'importance_type' : 'gain',
    'feature_fraction' : 0.6,
    'bagging_freq' : 5, 
    'bagging_fraction' : 0.9
}

model = LGBMClassifier(n_estimators=975, min_child_samples=5, max_depth=8, learning_rate=0.01,feature_fraction=0.6, 
                       bagging_freq=5, bagging_fraction= 0.9,objective ='binary

In [37]:
lgb_scores = []
predictions = 0

In [38]:
'''
param_grid = {'n_estimators': [100, 200, 400, 600],
              'learning_rate': [0.1, 0.05, 0.]}
grid = GridSearchCV(LGBMClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')
grid.fit(X, Y)
best_params = grid.best_params_
print('Best score of cross validation: {:.2f}'.format(grid.best_score_))
print('Best parameters:', best_params)
'''

"\nparam_grid = {'n_estimators': [100, 200, 400, 600],\n              'learning_rate': [0.1, 0.05, 0.]}\ngrid = GridSearchCV(LGBMClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')\ngrid.fit(X, Y)\nbest_params = grid.best_params_\nprint('Best score of cross validation: {:.2f}'.format(grid.best_score_))\nprint('Best parameters:', best_params)\n"

In [39]:
for fold, (idx_train, idx_valid) in enumerate(idx_fold):
    print(10*"=", f"Fold={fold+1}", 10*"=")
    
    X_train, X_valid = train.iloc[idx_train][col],train.iloc[idx_valid][col]
    Y_train, Y_valid = train.iloc[idx_train][TARGET],train.iloc[idx_valid][TARGET]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, Y_train, verbose = 0)
    
    # 정확도 측정해서 저장
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(Y_valid, preds_valid)
    lgb_scores.append(acc)
    
    preds = model.predict(test[col]) 
    predictions += preds/5
    
    print(f"Fold={fold+1}, Accuracy score : {acc :2f}%")
    
print("")
print("Mean Accuracy : ", np.mean(lgb_scores))

========== Fold=1 ==========
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Fold=1, Accuracy score : 0.817136%
========== Fold=2 ==========
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Fold=2, Accuracy score : 0.806210%
========== Fold=3 ==========
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature

---

---

# Submission

In [40]:
submission[TARGET] = predictions.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


# 결과
4/1 : accuracy : 0.791   |  등수 :   341 / 전체